<a href="https://colab.research.google.com/github/IsaacIndex/ISOM3260-CourseRegistration/blob/main/Isom3260.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Test commit
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ln -s /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle
! pip install shortuuid


--2023-04-01 14:53:50--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 23.205.72.79
Connecting to download.oracle.com (download.oracle.com)|23.205.72.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M   140MB/s    in 0.5s    

2023-04-01 14:53:51 (140 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

mv: cannot move 'instantclient_19_6' to '/opt/oracle/instantclient_19_6': Directory not empty
ln: failed to create symbolic link '/usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so': No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pu

In [ ]:
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "nhwongaa"
PASSWORD = "2392"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME) 
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns) 

c = conn.cursor()

In [27]:
from google.colab.widgets import TabBar
import sys
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from datetime import date
import shortuuid

In [29]:
#MANAGER
GLOBAL_MN_user_email = ""

# =====================================================
################  Tab Group 1 ################
# =====================================================
def MN_tabGroup1():
  MN_tabGroup1 = TabBar(['Login', 'Forget Password'])

  def showPage(tabName):
    with MN_tabGroup1.output_to(tabName):
      display()
#########################################################
################  Manager Login Tab - Begin ################
#########################################################
  with MN_tabGroup1.output_to('Login'):

  # create 6 label and 5 textboxes
    LOG_login_email_textbox = Text(description="Email: ")
    LOG_login_password_textbox = widgets.Password(description="Password: ")

    def get_login_password():
     p = ''
     typed = ''
     while True:
        typed = getch.getch()
        if typed == '\n':
            print(typed)
            break
        p += typed
     sys.stdout.write('*')
     sys.stdout.flush()

    LOG_status_label = widgets.HTML("")
    LOG_login_confirm_button = widgets.Button(description="Login", button_style="primary")
    LOG_login_forget_button = widgets.Button(description="Forget Password", button_style="info")

    def on_LOG_confirm_button_clicked(b):
      email = LOG_login_email_textbox.value
      password = LOG_login_password_textbox.value
      # data validation
      if ("@" not in email):
        LOG_status_label.value = "<span style='color: red'>Please enter a valid email</span>"
        return
      if (password == ""):
        LOG_status_label.value = "<span style='color: red'>Please enter your password</span>"
        return
      # fetch data
      c.execute("SELECT * FROM MANAGER WHERE MN_EMAIL = '{}' AND MN_PASSWORD = '{}'".format(email, password))
      row = c.fetchone()
      if row == None:
        LOG_status_label.value = "<span style='color: red'>Incorrect email or password</span>"
        LOG_login_password_textbox.value = ""
        return
      GLOBAL_MN_user_email = email
      LOG_status_label.value = "<span style='color: green'>Login Successful! Redirecting you to main page..</span>"
      time.sleep(2)
      MN_tabGroup1.remove()
      MN_tabGroup2()
    LOG_login_confirm_button.on_click(on_LOG_confirm_button_clicked)

    def on_LOG_login_forget_button_clicked(b):
      showPage("Forget Password")
    LOG_login_forget_button.on_click(on_LOG_login_forget_button_clicked)

    LOG_line1HBox = widgets.HBox([LOG_login_email_textbox])
    LOG_line2HBox = widgets.HBox([LOG_login_password_textbox])
    LOG_line3HBox = widgets.HBox([LOG_status_label])
    LOG_line4HBox = widgets.HBox([LOG_login_confirm_button, LOG_login_forget_button])

    display(widgets.VBox([LOG_line1HBox, LOG_line2HBox, LOG_line3HBox, LOG_line4HBox]))

################ Manager Login Tab - End ################

#########################################################
################  Manager Forget Password Tab - Begin ################
#########################################################
  with MN_tabGroup1.output_to('Forget Password', False):
    #create 2 label, 2 textboxes
    ForgetPassword_Label = Label('Forget Password')
    FP_MN_ID_Textbox = Text(value='',placeholder='Please Enter Your ID',description='Manager ID:',disabled=False)
    FP_Spacer = Label('----------------------------------------------------------------')
    FP_MN_Password_Textbox = Text(value='',placeholder='Your Password will appear here',description='Password',disabled=True)

    #Create 2 buttons
    FP_RetrievePW_btn = widgets.Button(description='Retrieve Pasword')
    FP_MainPage_btn = widgets.Button(description='Login')

    #define the events hadnlers for the two buttons when they are being clicked
    def on_FP_RetrievePW_btn_clicked(b):
      c.execute("SELECT * FROM STUDENT WHERE STU_ID = '" + FP_MN_ID_Textbox.value + "'")
      for row in c:
        if len(row[4]) >0:
          FP_MN_Password_Textbox.value = row[4]
        else:
          FP_MN_Password_Textbox.value = "Invalid ID"

    FP_RetrievePW_btn.on_click(on_FP_RetrievePW_btn_clicked)

    def on_FP_MainPage_btn_clicked(b):
      showPage('Login')
    FP_MainPage_btn.on_click(on_FP_MainPage_btn_clicked)

    #Display the objects
    FP_line1HBox = widgets.HBox([ForgetPassword_Label])
    FP_line2HBox = widgets.HBox([FP_MN_ID_Textbox])
    FP_line3HBox = widgets.HBox([FP_Spacer])
    FP_line4HBox = widgets.HBox([FP_MN_Password_Textbox])
    FP_line5HBox = widgets.HBox([FP_RetrievePW_btn,FP_MainPage_btn])

    display(widgets.VBox([FP_line1HBox,FP_line2HBox,FP_line3HBox,FP_line4HBox,FP_line5HBox]))
################ Student Forget Password Tab - End ################


# =====================================================
################  Tab Group 2 ################
# =====================================================
def MN_tabGroup2():
  MN_tabGroup2 = TabBar(['Main Page', 'Insert Student Data', 'Update Student Data', 'Delete Student Data','Retrieve Student Data', 'Insert Course Data', 'Update Course Data', 'Delete Course Data','Retrieve Course Data',], location="top")

  def showPage(tabName):
    with MN_tabGroup2.output_to(tabName):
      display()
#######################################################
################ Main Page Tab - Begin ################
#######################################################
  with MN_tabGroup2.output_to('Main Page'):

  # create 4 label 
    mainPage_label = Label("Please select a function...")
    mainPageStu_label = Label("STUDENT")
    UPD_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
    mainPageCou_label = Label("COURSE")

  # create 6 button
    btnInsertStuPage = widgets.Button(description="Insert Data")
    btnUpdateStuPage = widgets.Button(description="Update Data")
    btnDeleteStuPage = widgets.Button(description="Delete Data")
    btnRetrieveStuPage = widgets.Button(description="Retrieve Data")
    btnInsertCouPage = widgets.Button(description="Insert Data")
    btnUpdateCouPage = widgets.Button(description="Update Data")
    btnDeleteCouPage = widgets.Button(description="Delete Data")
    btnRetrieveCouPage = widgets.Button(description="Retrieve Data")

  # define the event handlers for the two buttons when they are being clicked
    def on_btnInsertStuPage_button_clicked(b):
      showPage("Insert Student Data")
    btnInsertStuPage.on_click(on_btnInsertStuPage_button_clicked)

    def on_btnUpdateStuPage_button_clicked(b):
      showPage("Update Student Data")
    btnUpdateStuPage.on_click(on_btnUpdateStuPage_button_clicked)

    def on_btnDeleteStuPage_button_clicked(b):
      showPage("Delete Student Data")
    btnDeleteStuPage.on_click(on_btnDeleteStuPage_button_clicked)

    def on_btnRetrieveStuPage_button_clicked(b):
      showPage("Retrieve Student Data")
    btnRetrieveStuPage.on_click(on_btnRetrieveStuPage_button_clicked)

    def on_btnInsertCouPage_button_clicked(b):
      showPage("Insert Course Data")
    btnInsertCouPage.on_click(on_btnInsertCouPage_button_clicked)

    def on_btnUpdateCouPage_button_clicked(b):
      showPage("Update Course Data")
    btnUpdateCouPage.on_click(on_btnUpdateCouPage_button_clicked)

    def on_btnDeleteCouPage_button_clicked(b):
      showPage("Delete Course Data")
    btnDeleteCouPage.on_click(on_btnDeleteCouPage_button_clicked)

    def on_btnRetrieveCouPage_button_clicked(b):
      showPage("Retrieve Course Data")
    btnRetrieveCouPage.on_click(on_btnRetrieveCouPage_button_clicked)

  # display the objects
    MainPage_row1HBox = widgets.HBox([mainPage_label])
    MainPage_row2HBox = widgets.HBox([mainPageStu_label])
    MainPage_row3HBox = widgets.HBox([btnInsertStuPage])
    MainPage_row4HBox = widgets.HBox([btnUpdateStuPage])
    MainPage_row5HBox = widgets.HBox([btnDeleteStuPage])
    MainPage_row6HBox = widgets.HBox([btnRetrieveStuPage])
    MainPage_row7HBox = widgets.HBox([UPD_spacer])
    MainPage_row8HBox = widgets.HBox([mainPageCou_label])
    MainPage_row9HBox = widgets.HBox([btnInsertCouPage])
    MainPage_row10HBox = widgets.HBox([btnUpdateCouPage])
    MainPage_row11HBox = widgets.HBox([btnDeleteCouPage])
    MainPage_row12HBox = widgets.HBox([btnRetrieveCouPage])


    display(widgets.VBox([MainPage_row1HBox, MainPage_row2HBox, MainPage_row3HBox, MainPage_row4HBox, MainPage_row5HBox, MainPage_row6HBox,MainPage_row7HBox, MainPage_row8HBox, MainPage_row9HBox, MainPage_row10HBox,MainPage_row11HBox,MainPage_row12HBox]))
################ Main Menu Tab - End ################


#########################################################
################ Insert Student Data Tab - Begin ################
#########################################################
  with MN_tabGroup2.output_to('Insert Student Data'):
    
    random_id = shortuuid.ShortUUID().random(length=8)
    
    INS_stu_id_label = Label("Student ID:")
    INS_stu_id_textbox = Text(value= random_id, disabled=True)
    
    INS_stu_name_label = Label("Name:")
    INS_stu_name_textbox = Text(placeholder='Name', disabled=False)

    INS_stu_address_label = Label("Address:")
    INS_stu_address_textbox = Text(placeholder='Address', disabled=False)

    INS_stu_email_label = Label("Email:")
    INS_stu_email_textbox = Text(placeholder='Email', disabled=False)
  
    INS_stu_password_label = Label("Password:")
    INS_stu_password_textbox = Text(placeholder='Password', disabled=False)
  
    INS_stu_current_job_label = Label("Current Job:")
    INS_stu_current_job_textbox = Text(placeholder='Current Job', disabled=False)
  
    INS_stu_qualification_label = Label("Qualification:")
    INS_stu_qualification_textbox = Text(placeholder='Qualification', disabled=False)

    INS_add_button = widgets.Button(description="Add Student")
    INS_clear_button = widgets.Button(description="Clear")
    INS_back_button = widgets.Button(description="Back")

    def on_INS_add_button_clicked(b):
      c.execute(
      """INSERT INTO STUDENT VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}')
      """.format(INS_stu_id_textbox.value, INS_stu_name_textbox.value, INS_stu_address_textbox.value, INS_stu_email_textbox.value, INS_stu_password_textbox.value, INS_stu_current_job_textbox.value, INS_stu_qualification_textbox.value))

      c.execute("commit")
      INS_add_button.disabled = True
      print("Done!")
    INS_add_button.on_click(on_INS_add_button_clicked)

    def on_INS_clear_button_clicked(b):
      INS_stu_id_textbox.value = ""
      INS_stu_name_textbox.value = ""
      INS_stu_address_textbox.value = ""
      INS_stu_email_textbox.value = ""
      INS_stu_password_textbox.value = ""
      INS_stu_current_job_textbox.value = ""
      INS_stu_qualification_textbox.value = ""
      INS_add_button.disabled = False
    INS_clear_button.on_click(on_INS_clear_button_clicked)

    def on_INS_back_button_clicked(b):
      showPage("Main Page")
    INS_back_button.on_click(on_INS_back_button_clicked)

  # display the objects
    INS_labelVBox = widgets.VBox([INS_stu_id_label, INS_stu_name_label, INS_stu_address_label, INS_stu_email_label, INS_stu_password_label, INS_stu_current_job_label, INS_stu_qualification_label])
    INS_textboxVBox = widgets.VBox([INS_stu_id_textbox, INS_stu_name_textbox, INS_stu_address_textbox, INS_stu_email_textbox, INS_stu_password_textbox, INS_stu_current_job_textbox, INS_stu_qualification_textbox])
    INS_line1HBox = widgets.HBox([INS_labelVBox, INS_textboxVBox])
    INS_line2HBox = widgets.HBox([INS_add_button, INS_clear_button])
    INS_line3HBox = widgets.HBox([INS_back_button])

    display(widgets.VBox([INS_line1HBox, INS_line2HBox, INS_line3HBox]))


################ Insert Student Data Tab - End ################


#########################################################
################ Insert Course Data Tab - Begin ################
#########################################################
  with MN_tabGroup2.output_to('Insert Course Data'):
    INS_crs_id_label = Label("Course ID:")
    INS_crs_id_textbox = Text(value='', placeholder='Enter Course ID', disabled=False)
    
    INS_crs_name_label = Label("Name:")
    INS_crs_name_textbox = Text(placeholder='Name', disabled=False)

    INS_crs_description_label = Label("Description:")
    INS_crs_description_textbox = Text(placeholder='Description', disabled=False)

    INS_crs_quota_label = Label("Quota:")
    INS_crs_quota_intbox = IntText(placeholder='Quota', disabled=False)
  
    INS_crs_timeslot_label = Label("Timeslot:")
    INS_crs_timeslot_datepicker = DatePicker(placeholder='Timeslot', disabled=False)
  
    INS_crs_enrollment_fee_label = Label("Enrollment fee:")
    INS_crs_enrollment_fee_intbox = IntText(placeholder='Enrollment Fee', disabled=False)

    INS_crs_status_label = Label("Enrollment fee:")
    INS_crs_status = widgets.Dropdown(
        options = ['Cancelled','Open for registration','Completed'],
        disabled = False
    )

    INS_instructor_ins_id_label = Label("Instructor ID:")
    INS_instructor_ins_id_textbox = Text(placeholder='Instructor ID', disabled=False)

    INS_add_button = widgets.Button(description="Add Course")
    INS_clear_button = widgets.Button(description="Clear")
    INS_back_button = widgets.Button(description="Back")

    def on_INS_add_button_clicked(b):
      c.execute(
      """INSERT INTO COURSE VALUES ('{}', '{}', '{}', {}, TO_DATE('{}', 'YYYY-MM-DD'), {}, '{}', '{}')
      """.format(INS_crs_id_textbox.value, INS_crs_name_textbox.value, INS_crs_description_textbox.value, INS_crs_quota_intbox.value, INS_crs_timeslot_datepicker.value, INS_crs_enrollment_fee_intbox.value, INS_crs_status.value, INS_instructor_ins_id_textbox.value))

      c.execute("commit")
      INS_add_button.disabled = True
      print("Done!")
    INS_add_button.on_click(on_INS_add_button_clicked)

    def on_INS_clear_button_clicked(b):
      INS_crs_id_textbox.value = ""
      INS_crs_name_textbox.value = ""
      INS_crs_description_textbox.value = ""
      INS_crs_quota_intbox.value = ""
      INS_crs_timeslot_datepicker.value = None
      INS_crs_enrollment_fee_intbox.value = ""
      INS_crs_status.value = ""
      INS_instructor_ins_id_textbox = ""
      INS_add_button.disabled = False
    INS_clear_button.on_click(on_INS_clear_button_clicked)

    def on_INS_back_button_clicked(b):
      showPage("Main Page")
    INS_back_button.on_click(on_INS_back_button_clicked)

  # display the objects
    INS_labelVBox = widgets.VBox([INS_crs_id_label, INS_crs_name_label, INS_crs_description_label, INS_crs_quota_label, INS_crs_timeslot_label, INS_crs_enrollment_fee_label, INS_crs_status_label, INS_instructor_ins_id_label])
    INS_textboxVBox = widgets.VBox([INS_crs_id_textbox, INS_crs_name_textbox, INS_crs_description_textbox, INS_crs_quota_intbox, INS_crs_timeslot_datepicker, INS_crs_enrollment_fee_intbox, INS_crs_status, INS_instructor_ins_id_textbox])
    INS_line1HBox = widgets.HBox([INS_labelVBox, INS_textboxVBox])
    INS_line2HBox = widgets.HBox([INS_add_button, INS_clear_button])
    INS_line3HBox = widgets.HBox([INS_back_button])

    display(widgets.VBox([INS_line1HBox, INS_line2HBox, INS_line3HBox]))
################ Insert Course Data Tab - End ################


#########################################################
################ Update Student Data Tab - Begin ################
#########################################################
  with MN_tabGroup2.output_to('Update Student Data'):

  # create 6 label and 5 textboxes
    UPD_stu_id_label = Label("Student ID: ")
    UPD_stu_id_textbox = Text(value='', placeholder='Enter Student ID', disabled=False)

    UPD_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

    UPD_stu_name_label = Label("Name: ")
    UPD_stu_name_textbox = Text(placeholder='Name', disabled=False)

    UPD_stu_email_label = Label("Email: ")
    UPD_stu_email_textbox = Text(placeholder='Email', disabled=False)

    UPD_stu_address_label = Label("Address: ")
    UPD_stu_address_textbox = Text(placeholder='Address', disabled=False)

    UPD_stu_password_label = Label("Password: ")
    UPD_stu_password_textbox = Text(placeholder='Password', disabled=False)

    UPD_stu_currentjob_label = Label("Current Job: ")
    UPD_stu_currentjob_textbox = Text(placeholder= 'Current Job', disabled=False)

    UPD_stu_qualification_label = Label("Qualification: ")
    UPD_stu_qualification_textbox = Text(placeholder='Qualification', disabled=False)

  # create 4 buttons
    UPD_retrieve_button = widgets.Button(description="Retrieve")
    UPD_update_button = widgets.Button(description="Update")
    UPD_clear_button = widgets.Button(description="Clear")
    UPD_back_button = widgets.Button(description="Back")

  # define the event handlers for the three buttons when they are being clicked
    def on_UPD_retrieve_button_clicked(b):
      c.execute("SELECT * FROM STUDENT WHERE STU_ID = '" + UPD_stu_id_textbox.value + "'")
      for row in c:
        UPD_stu_name_textbox.value = row[1]
        UPD_stu_address_textbox.value = row[2]
        UPD_stu_email_textbox.value = row[3]
        UPD_stu_password_textbox.value = row[4]
        UPD_stu_currentjob_textbox.value = row[5]
        UPD_stu_qualification_textbox.value = row[6]
    UPD_retrieve_button.on_click(on_UPD_retrieve_button_clicked)
    
    def on_UPD_clear_button_clicked(b):
      UPD_stu_name_textbox.value = ""
      UPD_stu_address_textbox.value = ""
      UPD_stu_email_textbox.value = ""
      UPD_stu_password_textbox.value = ""
      UPD_stu_currentjob_textbox.value = ""
      UPD_stu_qualification_textbox.value = ""
    UPD_clear_button.on_click(on_UPD_clear_button_clicked)  
      
    def on_UPD_update_button_clicked(b):
      c.execute(
      """
        UPDATE STUDENT 
        SET STU_NAME = '{}',
        STU_ADDRESS = '{}',
        STU_EMAIL = '{}',
        STU_PASSWORD = '{}',
        STU_CURRENTJOB = '{}',
        STU_QUALIFICATION = '{}'
        WHERE STU_ID = '{}'
      """.format(UPD_stu_name_textbox.value, UPD_stu_address_textbox.value, UPD_stu_email_textbox.value, UPD_stu_password_textbox.value, UPD_stu_currentjob_textbox.value, UPD_stu_qualification_textbox.value, UPD_stu_id_textbox.value)
      )
      c.execute("commit")
      print("Update Success!")
    UPD_update_button.on_click(on_UPD_update_button_clicked)

    def on_UPD_back_button_clicked(b):
      showPage("Main Page")
    UPD_back_button.on_click(on_UPD_back_button_clicked)

  # display the objects
    UPD_labelVBox = widgets.VBox([UPD_stu_name_label, UPD_stu_address_label, UPD_stu_email_label, UPD_stu_password_label, UPD_stu_currentjob_label, UPD_stu_qualification_label])
    UPD_textboxVBox = widgets.VBox([UPD_stu_name_textbox, UPD_stu_address_textbox, UPD_stu_email_textbox, UPD_stu_password_textbox, UPD_stu_currentjob_textbox, UPD_stu_qualification_textbox])
    
    UPD_line1HBox = widgets.HBox([UPD_stu_id_label, UPD_stu_id_textbox, UPD_retrieve_button])
    UPD_line2HBox = widgets.HBox([UPD_spacer])
    UPD_line3HBox = widgets.HBox([UPD_labelVBox, UPD_textboxVBox])
    UPD_line4HBox = widgets.HBox([UPD_update_button, UPD_clear_button])
    UPD_line5HBox = widgets.HBox([UPD_back_button])

    display(widgets.VBox([UPD_line1HBox, UPD_line2HBox, UPD_line3HBox, UPD_line4HBox, UPD_line5HBox]))


################ Update Student Data Tab - End ################

#########################################################
################ Update Course Data Tab - Begin ################
#########################################################
  with MN_tabGroup2.output_to('Update Course Data'):

  # create 7 label and 5 textboxes
    UPD_crs_id_label = Label("Course ID: ")
    UPD_crs_id_textbox = Text(value='', placeholder='Enter Course ID', disabled=False)

    UPD_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

    UPD_crs_name_label = Label("Name: ")
    UPD_crs_name_textbox = Text(placeholder='Name', disabled=False)

    UPD_crs_desc_label = Label("Description: ")
    UPD_crs_desc_textbox = Text(placeholder='Description', disabled=False)

    UPD_crs_quota_label = Label("Quota: ")
    UPD_crs_quota_intbox = IntText(placeholder='Quota', disabled=False)

    UPD_crs_timeslot_label = Label("Timeslot: ") 
    UPD_crs_timeslot_datepicker = DatePicker(dplaceholder='Timeslot', disabled=False) 

    UPD_crs_enrolfee_label = Label("Enrollment Fee: ") 
    UPD_crs_enrolfee_intbox = IntText(placeholder='Enrollment Fee', disabled=False) 

    UPD_crs_status_label = Label("Status: ")
    UPD_crs_status_textbox = Text(placeholder='Status', disabled=False)

    UPD_crs_instructor_label = Label("Instructor ID: ")
    UPD_crs_instructor_textbox = Text(placeholder='Instructor ID', disabled=False)

    
  # create 4 buttons
    UPD_retrieve_button = widgets.Button(description="Retrieve")
    UPD_update_button = widgets.Button(description="Update")
    UPD_clear_button = widgets.Button(description="Clear")
    UPD_back_button = widgets.Button(description="Back")

  # define the event handlers for the three buttons when they are being clicked
    def on_UPD_retrieve_button_clicked(b):
      c.execute("SELECT * FROM COURSE WHERE CRS_ID = '" + UPD_crs_id_textbox.value + "'")
      for row in c:
        UPD_crs_name_textbox.value = row[1]
        UPD_crs_desc_textbox.value = row[2]
        UPD_crs_quota_intbox.value = row[3]
        UPD_crs_timeslot_datepicker.value = row[4] ####TIMESLOTTTTTTTT
        UPD_crs_enrolfee_intbox.value = row[5] ####### ENROLLMENT FEE INT BOX?
        UPD_crs_status_textbox.value = row[6]
        UPD_crs_instructor_textbox.value = row[7]
    UPD_retrieve_button.on_click(on_UPD_retrieve_button_clicked)
    
    def on_UPD_clear_button_clicked(b):
      UPD_crs_name_textbox.value = ""
      UPD_crs_desc_textbox.value = ""
      UPD_crs_quota_intbox.value = 0 
      UPD_crs_timeslot_datepicker.value = None
      UPD_crs_enrolfee_intbox.value = 0 
      UPD_crs_status_textbox.value = ""
      UPD_crs_instructor_textbox.value = ""
    UPD_clear_button.on_click(on_UPD_clear_button_clicked)  
      
    def on_UPD_update_button_clicked(b):
      c.execute(
      """
        UPDATE COURSE 
        SET CRS_NAME = '{}',
        CRS_DESC = '{}',
        CRS_QUOTA = {},
        CRS_TIMESLOT = TO_DATE('{}', 'YYYY-MM-DD'),
        CRS_ENROLLMENT_FEE = {},
        CRS_STATUS = '{}',
        INSTRUCTOR_INS_ID ='{}',
        WHERE CRS_ID = '{}'
      """.format(UPD_crs_name_textbox.value, UPD_crs_desc_textbox.value, UPD_crs_quota_intbox.value, UPD_crs_timeslot_datepicker.value, UPD_crs_enrolfee_intbox.value, UPD_crs_status_textbox.value, UPD_crs_instructor_textbox.value, UPD_crs_id_textbox.value))
      c.execute("commit")
      print("Update Success!")
    UPD_update_button.on_click(on_UPD_update_button_clicked)

    def on_UPD_back_button_clicked(b):
      showPage("Main Page")
    UPD_back_button.on_click(on_UPD_back_button_clicked)
    

  # display the objects
    UPD_labelVBox = widgets.VBox([UPD_crs_name_label, UPD_crs_desc_label, UPD_crs_quota_label, UPD_crs_timeslot_label, UPD_crs_enrolfee_label, UPD_crs_status_label, UPD_crs_instructor_label])
    UPD_textboxVBox = widgets.VBox([UPD_crs_name_textbox, UPD_crs_desc_textbox, UPD_crs_quota_intbox, UPD_crs_timeslot_datepicker, UPD_crs_enrolfee_intbox, UPD_crs_status_textbox, UPD_crs_instructor_textbox])

    UPD_line1HBox = widgets.HBox([UPD_crs_id_label, UPD_crs_id_textbox, UPD_retrieve_button])
    UPD_line2HBox = widgets.HBox([UPD_spacer])
    UPD_line3HBox = widgets.HBox([UPD_labelVBox, UPD_textboxVBox])
    UPD_line4HBox = widgets.HBox([UPD_update_button, UPD_clear_button])
    UPD_line5HBox = widgets.HBox([UPD_back_button])

    display(widgets.VBox([UPD_line1HBox, UPD_line2HBox, UPD_line3HBox, UPD_line4HBox, UPD_line5HBox]))

################ Update Course Data Tab - End ################

#########################################################
################  Retrieve Student Data Tab - Begin ################
#########################################################
  with MN_tabGroup2.output_to('Retrieve Student Data'):

    RET_stu_id_label = Label("Student ID:")
    RET_stu_id_textbox = Text(value='', placeholder='Enter Student ID', disabled=False)
    
    spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  
    RET_stu_name_label = Label("Name:")
    RET_stu_name_textbox = Text(placeholder='Name', disabled=False)

    RET_stu_address_label = Label("Address:")
    RET_stu_address_textbox = Text(placeholder='Address', disabled=False)

    RET_stu_email_label = Label("Email:")
    RET_stu_email_textbox = Text(placeholder='Email', disabled=False)
  
    RET_stu_password_label = Label("Password:")
    RET_stu_password_textbox = Text(placeholder='Password', disabled=False)
  
    RET_stu_current_job_label = Label("Current Job:")
    RET_stu_current_job_textbox = Text(placeholder='Current Job', disabled=False)
  
    RET_stu_qualification_label = Label("Qualification:")
    RET_stu_qualification_textbox = Text(placeholder='Qualification', disabled=False)

  # create buttons
    RET_stu_retrieve_button = widgets.Button(description="Retrieve")
    RET_stu_clear_button = widgets.Button(description="Clear")
    RET_stu_back_button = widgets.Button(description="Back")

  # define the event handlers for the two buttons when they are being clicked
    def on_RET_stu_retrieve_button_clicked(b):
      c.execute("SELECT * FROM STUDENT WHERE STU_ID = '" + RET_stu_id_textbox.value + "'")
      for row in c:
        RET_stu_name_textbox.value = row[1]
        RET_stu_address_textbox.value = row[2]
        RET_stu_email_textbox.value = row[3]
        RET_stu_password_textbox.value = row[4]
        RET_stu_current_job_textbox.value = row[5]
        RET_stu_qualification_textbox.value = row[6]
    RET_stu_retrieve_button.on_click(on_RET_stu_retrieve_button_clicked)

    def on_RET_stu_clear_button_clicked(b):
      RET_stu_id_textbox.value = ""
      RET_stu_name_textbox.value = ""
      RET_stu_address_textbox.value = ""
      RET_stu_email_textbox.value = ""
      RET_stu_password_textbox.value = ""
      RET_stu_current_job_textbox.value = ""
      RET_stu_qualification_textbox.value = ""
    RET_stu_clear_button.on_click(on_RET_stu_clear_button_clicked)

    def on_RET_stu_back_button_clicked(b):
      showPage("Main Page")
    RET_stu_back_button.on_click(on_RET_stu_back_button_clicked)

  # display the objects

    RET_stu_labelVBox = widgets.VBox([RET_stu_name_label,RET_stu_address_label,RET_stu_email_label,RET_stu_password_label,RET_stu_current_job_label,RET_stu_qualification_label])
    RET_stu_textboxVBox = widgets.VBox([RET_stu_name_textbox,RET_stu_address_textbox,RET_stu_email_textbox,RET_stu_password_textbox,RET_stu_current_job_textbox,RET_stu_qualification_textbox])
    RET_stu_line1HBox = widgets.HBox([RET_stu_id_label,RET_stu_id_textbox, RET_stu_retrieve_button])
    RET_stu_line2HBox = widgets.HBox([spacer])
    RET_stu_line3HBox = widgets.HBox([RET_stu_labelVBox, RET_stu_textboxVBox])
    RET_stu_line4HBox = widgets.HBox([RET_stu_clear_button])
    RET_stu_line5HBox = widgets.HBox([RET_stu_back_button])
    
    display(widgets.VBox([RET_stu_line1HBox, RET_stu_line2HBox, RET_stu_line3HBox,RET_stu_line4HBox, RET_stu_line5HBox]))


################ Retrieve Student Data Tab - End ################

####################################################
################  Retrieve Course Data Tab - Begin ################
#########################################################
  with MN_tabGroup2.output_to('Retrieve Course Data'):

    RET_crs_id_label = Label("Course ID:")
    RET_crs_id_textbox = Text(value='', placeholder='Enter Course ID', disabled=False)
    
    spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  
    RET_crs_name_label = Label("Name:")
    RET_crs_name_textbox = Text(placeholder='Name', disabled=False)

    RET_crs_description_label = Label("Description:")
    RET_crs_description_textbox = Text(placeholder='Description', disabled=False)

    RET_crs_quota_label = Label("Quota:")
    RET_crs_quota_intbox = IntText(placeholder='Quota', disabled=False)
  
    RET_crs_timeslot_label = Label("Timeslot:")
    RET_crs_timeslot_datepicker = DatePicker(placeholder='Timeslot', disabled=False)
  
    RET_crs_enrollment_fee_label = Label("Enrollment Fee:") 
    RET_crs_enrollment_fee_intbox = Text(placeholder='Enrollment Fee', disabled=False)
  
    RET_crs_status_label = Label("Status:")
    RET_crs_status_textbox = Text(placeholder='Status', disabled=False)

    RET_crs_instructor_label = Label("Instructor ID: ")
    RET_crs_instructor_textbox = Text(placeholder='Instructor ID ', disabled=False)

  # create buttons
    RET_crs_retrieve_button = widgets.Button(description="Retrieve")
    RET_crs_clear_button = widgets.Button(description="Clear")
    RET_crs_back_button = widgets.Button(description="Back")

  # define the event handlers for the two buttons when they are being clicked
    def on_RET_crs_retrieve_button_clicked(b):
      c.execute("SELECT * FROM COURSE WHERE CRS_ID = '" + RET_crs_id_textbox.value + "'")
      for row in c:
        RET_crs_name_textbox.value = row[1]
        RET_crs_description_textbox.value = row[2]
        RET_crs_quota_intbox.value = row[3]
        RET_crs_timeslot_datepicker.value = row[4]
        RET_crs_enrollment_fee_intbox.value = row[5]
        RET_crs_status_textbox.value = row[6]
        RET_crs_instructor_textbox = row[7]
    RET_crs_retrieve_button.on_click(on_RET_crs_retrieve_button_clicked)
 
    def on_RET_crs_clear_button_clicked(b):
      RET_crs_id_textbox.value = ""
      RET_crs_name_textbox.value = ""
      RET_crs_description_textbox.value = ""
      RET_crs_quota_intbox.value = 0
      RET_crs_timeslot_datepicker.value = None
      RET_crs_enrollment_fee_intbox.value = 0
      RET_crs_status_textbox.value = ""
      RET_crs_instructor_textbox = ""
    RET_crs_clear_button.on_click(on_RET_crs_clear_button_clicked)

    def on_RET_crs_back_button_clicked(b):
      showPage("Main Page")
    RET_crs_back_button.on_click(on_RET_crs_back_button_clicked)

  # display the objects

    RET_crs_labelVBox = widgets.VBox([RET_crs_name_label,RET_crs_description_label,RET_crs_quota_label,RET_crs_timeslot_label,RET_crs_enrollment_fee_label,RET_crs_status_label,RET_crs_instructor_label])
    RET_crs_textboxVBox = widgets.VBox([RET_crs_name_textbox,RET_crs_description_textbox,RET_crs_quota_intbox,RET_crs_timeslot_datepicker,RET_crs_enrollment_fee_intbox,RET_crs_status_textbox,RET_crs_instructor_textbox])
    RET_line1HBox = widgets.HBox([RET_crs_id_label,RET_crs_id_textbox, RET_crs_retrieve_button])
    RET_line2HBox = widgets.HBox([spacer])
    RET_line3HBox = widgets.HBox([RET_crs_labelVBox, RET_crs_textboxVBox])
    RET_line4HBox = widgets.HBox([RET_crs_clear_button])
    RET_line5HBox = widgets.HBox([RET_crs_back_button])
    
    display(widgets.VBox([RET_line1HBox, RET_line2HBox, RET_line3HBox,RET_line4HBox, RET_line5HBox]))

################ Retrieve Course Data Tab - End ################

#########################################################
################  Delete Student Data Tab - Begin ################
#########################################################
  with MN_tabGroup2.output_to('Delete Student Data'):

    DEL_stu_id_label = Label("Student ID: ")
    DEL_stu_id_textbox = Text(value='', placeholder='Enter Student ID', disabled=False)

    DEL_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

    DEL_stu_name_label = Label("Name: ")
    DEL_stu_name_textbox = Text(placeholder='Name', disabled=True)

    DEL_stu_address_label = Label("Address: ")
    DEL_stu_address_textbox = Text(placeholder='Address', disabled=True)

    DEL_stu_email_label = Label("Email: ")
    DEL_stu_email_textbox = Text(placeholder='Email', disabled=True)

    DEL_stu_password_label = Label("Password: ")
    DEL_stu_password_textbox = Text(placeholder='Password', disabled=True)

    DEL_stu_current_job_label = Label("Current Job: ")
    DEL_stu_current_job_textbox = Text(placeholder='Current Job', disabled=True)

    DEL_stu_qualification_label = Label("Qualification: ")
    DEL_stu_qualification_textbox = Text(placeholder='Qualification', disabled=True)
    


# create 4 buttons
    DEL_retrieve_button = widgets.Button(description="Retrieve")
    DEL_delete_button = widgets.Button(description="Delete")
    DEL_clear_button = widgets.Button(description="Clear")
    DEL_back_button = widgets.Button(description="Back")

# define the event handlers for the three buttons when they are being clicked
    def on_DEL_retrieve_button_clicked(b):
      c.execute("SELECT * FROM STUDENT WHERE STU_ID = '" + DEL_stu_id_textbox.value + "'")
      for row in c:
        DEL_stu_name_textbox.value = row[1]
        DEL_stu_address_textbox.value = row[2]
        DEL_stu_email_textbox.value = row[3]
        DEL_stu_password_textbox.value = row[4]
        DEL_stu_current_job_textbox.value = row[5]
        DEL_stu_qualification_textbox.value = row[6]
    DEL_retrieve_button.on_click(on_DEL_retrieve_button_clicked)
  
    def on_DEL_clear_button_clicked(b):
      DEL_stu_name_textbox.value = ""
      DEL_stu_address_textbox.value = ""
      DEL_stu_email_textbox.value = ""
      DEL_stu_password_textbox.value = ""
      DEL_stu_current_job_textbox.value = ""
      DEL_stu_qualification_textbox.value = ""

    DEL_clear_button.on_click(on_DEL_clear_button_clicked)  
    
    def on_DEL_delete_button_clicked(b):
      c.execute(
    """
          DELETE FROM STUDENT 
          WHERE STU_ID = '{}'
    """.format(DEL_stu_id_textbox.value)
    )
      c.execute("commit")
      print("Delete Success!")
    DEL_delete_button.on_click(on_DEL_delete_button_clicked)

    def on_DEL_back_button_clicked(b):
      showPage("Main Page")
    DEL_back_button.on_click(on_DEL_back_button_clicked)

# display the objects
    DEL_textboxVBox = widgets.VBox([DEL_stu_name_textbox, DEL_stu_address_textbox, DEL_stu_email_textbox, DEL_stu_password_textbox, DEL_stu_current_job_textbox, DEL_stu_qualification_textbox])
    DEL_labelVBox = widgets.VBox([DEL_stu_name_label,DEL_stu_address_label,DEL_stu_email_label, DEL_stu_password_label, DEL_stu_current_job_label, DEL_stu_qualification_label])

    DEL_line1HBox = widgets.HBox([DEL_stu_id_label, DEL_stu_id_textbox, DEL_retrieve_button])
    DEL_line2HBox = widgets.HBox([DEL_spacer])
    DEL_line3HBox = widgets.HBox([DEL_labelVBox, DEL_textboxVBox])
    DEL_line4HBox = widgets.HBox([DEL_delete_button, DEL_clear_button])
    DEL_line5HBox = widgets.HBox([DEL_back_button])
  
    display(widgets.VBox([DEL_line1HBox, DEL_line2HBox, DEL_line3HBox, DEL_line4HBox, DEL_line5HBox]))


################ Delete Student Data Tab - End ################


#####

####################################################
################  Delete Course Data Tab - Begin ################
#########################################################
  with MN_tabGroup2.output_to('Delete Course Data'):

    DEL_crs_id_label = Label("Course ID: ")
    DEL_crs_id_textbox = Text(value='', placeholder='Enter Course ID', disabled=False)

    DEL_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

    DEL_crs_name_label = Label("Name: ")
    DEL_crs_name_textbox = Text(placeholder='Name', disabled=True)

    DEL_crs_description_label = Label("Description: ")
    DEL_crs_description_textbox = Text(placeholder='Description', disabled=True)

    DEL_crs_quota_label = Label("Quota: ")
    DEL_crs_quota_intbox = IntText(placeholder='Quota', disabled=True)

    DEL_crs_timeslot_label = Label("Timeslot: ")
    DEL_crs_timeslot_datepicker = DatePicker(placeholder='Timeslot', disabled=True)

    DEL_crs_enrollment_fee_label = Label("Enrollment fee: ")
    DEL_crs_enrollment_fee_intbox = IntText(placeholder='Enrollment Fee', disabled=True)

    DEL_crs_status_label = Label("Status: ")
    DEL_crs_status_textbox = Text(placeholder='Status:', disabled=True)

    DEL_instructor_ins_id_label = Label("Instructor ID: ")
    DEL_instructor_ins_id_textbox = Text(placeholder='Instructor ID', disabled=True)

# create 4 buttons
    DEL_retrieve_button = widgets.Button(description="Retrieve")
    DEL_delete_button = widgets.Button(description="Delete")
    DEL_clear_button = widgets.Button(description="Clear")
    DEL_back_button = widgets.Button(description="Back")

# define the event handlers for the three buttons when they are being clicked
    def on_DEL_retrieve_button_clicked(b):
      c.execute("SELECT * FROM COURSE WHERE CRS_ID = '" + DEL_crs_id_textbox.value + "'")
      for row in c:
        DEL_crs_name_textbox.value = row[1]
        DEL_crs_description_textbox.value = row[2]
        DEL_crs_quota_intbox.value = row[3]
        DEL_crs_timeslot_datepicker.value = row[4]
        DEL_crs_enrollment_fee_intbox.value = row[5]
        DEL_crs_status_textbox.value = row[6]
        DEL_instructor_ins_id_textbox.value = row[7]
    DEL_retrieve_button.on_click(on_DEL_retrieve_button_clicked)
  
    def on_DEL_clear_button_clicked(b):
      DEL_crs_name_textbox.value = ""
      DEL_crs_description_textbox.value = ""
      DEL_crs_quota_intbox.value = ""
      DEL_crs_timeslot_datepicker.value = ""
      DEL_crs_enrollment_fee_intbox.value = ""
      DEL_crs_status_textbox.value = ""
      DEL_instructor_ins_id_textbox.value = ""

    DEL_clear_button.on_click(on_DEL_clear_button_clicked)  
    
    def on_DEL_delete_button_clicked(b):
      c.execute(
    """ DELETE FROM COURSE WHERE CRS_ID = '{}'""".format(DEL_crs_id_textbox.value)
    )
      c.execute("commit")
      print("Delete Success!")
    DEL_delete_button.on_click(on_DEL_delete_button_clicked)

    def on_DEL_back_button_clicked(b):
      showPage("Main Page")
    DEL_back_button.on_click(on_DEL_back_button_clicked)

# display the objects
    DEL_textboxVBox = widgets.VBox([DEL_crs_name_textbox, DEL_crs_description_textbox, DEL_crs_quota_intbox, DEL_crs_timeslot_datepicker, DEL_crs_enrollment_fee_intbox, DEL_crs_status_textbox, DEL_instructor_ins_id_textbox])
    DEL_labelVBox = widgets.VBox([DEL_crs_name_label,DEL_crs_description_label, DEL_crs_quota_label, DEL_crs_timeslot_label, DEL_crs_enrollment_fee_label, DEL_crs_status_label, DEL_instructor_ins_id_label])

    DEL_line1HBox = widgets.HBox([DEL_crs_id_label, DEL_crs_id_textbox, DEL_retrieve_button])
    DEL_line2HBox = widgets.HBox([DEL_spacer])
    DEL_line3HBox = widgets.HBox([DEL_labelVBox, DEL_textboxVBox])
    DEL_line4HBox = widgets.HBox([DEL_delete_button, DEL_clear_button])
    DEL_line5HBox = widgets.HBox([DEL_back_button])
  
    display(widgets.VBox([DEL_line1HBox, DEL_line2HBox, DEL_line3HBox, DEL_line4HBox, DEL_line5HBox]))


################ Delete Course Data Tab - End ################


MN_tabGroup2()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#STUDENT
GLOBAL_STU_user_id = ""
GLOBAL_STU_user_name = ""
GLOBAL_STU_user_address = ""
GLOBAL_STU_user_email = ""
GLOBAL_STU_user_currentJob = ""
GLOBAL_STU_user_qualification = ""

# =====================================================
################  Tab Group 1 ################
# =====================================================
def STU_tabGroup1():
  STU_tabGroup1 = TabBar(['Login', 'Forget Password', 'Student Registration'])

  def showPage(tabName):
    with STU_tabGroup1.output_to(tabName):
      display()
#########################################################
################  Student Login Tab - Begin ################
#########################################################
  with STU_tabGroup1.output_to('Login'):

  # create 6 label and 5 textboxes
    LOG_login_email_textbox = Text(description="Email: ")
    LOG_login_password_textbox = widgets.Password(description="Password: ")
    LOG_status_label = widgets.HTML("")
    LOG_login_confirm_button = widgets.Button(description="Login", button_style="primary")
    LOG_login_forget_button = widgets.Button(description="Forget Password", button_style="info")

    def on_LOG_confirm_button_clicked(b):
      email = LOG_login_email_textbox.value
      password = LOG_login_password_textbox.value
      # data validation
      if ("@" not in email):
        LOG_status_label.value = "<span style='color: red'>Please enter a valid email</span>"
        return
      if (password == ""):
        LOG_status_label.value = "<span style='color: red'>Please enter your password</span>"
        return
      # fetch data
      c.execute("SELECT * FROM STUDENT WHERE STU_EMAIL = '{}' AND STU_PASSWORD = '{}'".format(email, password))
      row = c.fetchone()
      if row == None:
        LOG_status_label.value = "<span style='color: red'>Incorrect email or password</span>"
        LOG_login_password_textbox.value = ""
        return
      global GLOBAL_STU_user_id, GLOBAL_STU_user_name, GLOBAL_STU_user_address, GLOBAL_STU_user_email, GLOBAL_STU_user_currentJob, GLOBAL_STU_user_qualification
      GLOBAL_STU_user_id = row[0]
      GLOBAL_STU_user_name = row[1]
      GLOBAL_STU_user_address = row[2]
      GLOBAL_STU_user_email = row[3]
      GLOBAL_STU_user_currentJob = row[5]
      GLOBAL_STU_user_qualification = row[6]
      # print(GLOBAL_STU_user_id,GLOBAL_STU_user_name,GLOBAL_STU_user_address,GLOBAL_STU_user_email,GLOBAL_STU_user_currentJob,GLOBAL_STU_user_qualification)
      LOG_status_label.value = "<span style='color: green'>Login Successful! Redirecting you to main page..</span>"
      time.sleep(2)
      STU_tabGroup1.remove()
      STU_tabGroup2()
    LOG_login_confirm_button.on_click(on_LOG_confirm_button_clicked)

    def on_LOG_login_forget_button_clicked(b):
      showPage("Forget Password")
    LOG_login_forget_button.on_click(on_LOG_login_forget_button_clicked)

    LOG_line1HBox = widgets.HBox([LOG_login_email_textbox])
    LOG_line2HBox = widgets.HBox([LOG_login_password_textbox])
    LOG_line3HBox = widgets.HBox([LOG_status_label])
    LOG_line4HBox = widgets.HBox([LOG_login_confirm_button, LOG_login_forget_button])

    display(widgets.VBox([LOG_line1HBox, LOG_line2HBox, LOG_line3HBox, LOG_line4HBox]))
################ Student Login Tab - End ################


#########################################################
################  Student Forget Password Tab - Begin ################
#########################################################
  with STU_tabGroup1.output_to('Forget Password', False):
    #create 2 label, 2 textboxes
    ForgetPassword_Label = Label('Forget Password')
    FP_StuID_Textbox = Text(value='',placeholder='Please Enter Your ID',description='Student ID:',disabled=False)
    FP_Spacer = Label('----------------------------------------------------------------')
    FP_StuPassword_Textbox = Text(value='',placeholder='Your Password will appear here',description='Password',disabled=True)

    #Create 2 buttons
    FP_RetrievePW_btn = widgets.Button(description='Retrieve Pasword')
    FP_MainPage_btn = widgets.Button(description='Main Page')

    #define the events hadnlers for the two buttons when they are being clicked
    def on_FP_RetrievePW_btn_clicked(b):
      c.execute("SELECT * FROM STUDENT WHERE STU_ID = '" + FP_StuID_Textbox.value + "'")
      for row in c:
        if len(row[4]) >0:
          FP_StuPassword_Textbox.value = row[4]
        else:
          FP_StuPassword_Textbox.value = "Invalid ID"

    FP_RetrievePW_btn.on_click(on_FP_RetrievePW_btn_clicked)

    def on_FP_MainPage_btn_clicked(b):
      showPage('Main Page')
    FP_MainPage_btn.on_click(on_FP_MainPage_btn_clicked)

    #Display the objects
    FP_line1HBox = widgets.HBox([ForgetPassword_Label])
    FP_line2HBox = widgets.HBox([FP_StuID_Textbox])
    FP_line3HBox = widgets.HBox([FP_Spacer])
    FP_line4HBox = widgets.HBox([FP_StuPassword_Textbox])
    FP_line5HBox = widgets.HBox([FP_RetrievePW_btn,FP_MainPage_btn])

    display(widgets.VBox([FP_line1HBox,FP_line2HBox,FP_line3HBox,FP_line4HBox,FP_line5HBox]))
################ Student Forget Password Tab - End ################


#########################################################
################  Student Registration Tab - Begin ################
#########################################################
  with STU_tabGroup1.output_to('Student Registration', False):
    #create 4 textboxes , 1 label and 1 selectbox
    UserReg_Label = Label('User Registration')
    Email_Textbox = Text(value='',placeholder='Please Enter Email Address',description='Email:',disabled=False)
    Password_Textbox = Text(value='', placeholder='Please Enter Password',description="Password",disable=False)
    #Confirm_Textbox = Text(value='',Placeholder = 'Please Re-enter Password',description='Email',disable=False)
    Name_Textbox = Text(value='',Placeholder = 'Please Enter Your Name',description='Name',disable=False)
    Occupation_Textbox = Text(value='',Placeholder = 'Please Enter Your Occupation',description='Occupation',disable=False)
    Address_Textbox = Text(value='',Placeholder = 'Please Enter Your Address',description='Address',disable=False)
    Qualification_Selectbox = widgets.Dropdown(
      options=['Please Choose Your highest qualification','Primary School or Below','Secondary School','Higher Diploma/Associate Degree','Bachelor  Degree or Above'],
      value='Please Choose Your highest qualification',
      description='Highest Qualification:',
      disabled=False,)
    #create 2 buttons
    StuReg_Reg_btn = widgets.Button(description = 'Register')
    StuReg_Cancel_btn = widgets.Button(description='Cancel')

    #Define the random ID that will be used as the stu_id here
    random_id = shortuuid.ShortUUID().random(length=8)


    #define action to be taken when the register and cancel button are clicked
    def on_StuReg_Reg_btn_clicked(b):
      c.execute(
      """INSERT INTO STUDENT VALUES ('{}', '{}', '{}', '{}', '{}','{}','{}')
      """.format(random_id, Name_Textbox.value, Address_Textbox.value, Email_Textbox.value, Password_Textbox.value,Occupation_Textbox.value,Qualification_Selectbox.value))

      c.execute("commit")
      ##Those variable for mainpage will be assigned here (waiting for Issac)
      #showPage('')
      print(f'''
      
      This is your unique ID, Please remember that: {random_id}
      
      ''')
    StuReg_Reg_btn.on_click(on_StuReg_Reg_btn_clicked)

    def on_StuReg_Cancel_btn_clicked(b):
      showPage('')
    
    StuReg_Cancel_btn.on_click(on_StuReg_Cancel_btn_clicked)

      #display the objects
    StuReg_row1HBox = widgets.HBox([UserReg_Label])
    StuReg_row2HBox = widgets.HBox([Email_Textbox])
    StuReg_row3HBox = widgets.HBox([Password_Textbox])
    StuReg_row4HBox = widgets.HBox([Name_Textbox])
    StuReg_row5HBox = widgets.HBox([Occupation_Textbox])
    StuReg_row6HBox = widgets.HBox([Address_Textbox])
    StuReg_row7HBox = widgets.HBox([Qualification_Selectbox])
    StuReg_row8HBox = widgets.HBox([StuReg_Reg_btn,StuReg_Cancel_btn])
    display(widgets.VBox([StuReg_row1HBox,StuReg_row2HBox,StuReg_row3HBox,StuReg_row4HBox,StuReg_row5HBox,StuReg_row6HBox,StuReg_row7HBox,StuReg_row8HBox]))
################ Student Registration Tab - End ################


# =====================================================
################  Tab Group 2 ################
# =====================================================
def STU_tabGroup2():
  STU_tabGroup2 = TabBar(['Update Info', 'Test'], "top")

  def showPage(tabName):
    with STU_tabGroup2.output_to(tabName):
      display()
#########################################################
################  Student Main Tab - Begin ################
#########################################################
  with STU_tabGroup2.output_to('Update Info'):
    UP_id = Text(description="id", value=GLOBAL_STU_user_id, disabled=True)
    UP_name = Text(description="name", value=GLOBAL_STU_user_name, disabled=False)
    UP_address = Text(description="address", value=GLOBAL_STU_user_address, disabled=False)
    UP_email = Text(description="email", value=GLOBAL_STU_user_email, disabled=False)
    UP_currentJob = Text(description="Job", value=GLOBAL_STU_user_currentJob, disabled=False)
    UP_qualification = Text(description="qualification", value=GLOBAL_STU_user_qualification, disabled=False)
    UP_status_label = widgets.HTML("")

    UP_update_button = widgets.Button(description="Update")

    def on_UP_update_button_clicked(b):
      try:
        c.execute("UPDATE STUDENT SET STU_NAME = '{}', STU_ADDRESS = '{}', STU_EMAIL = '{}', STU_CURRENTJOB = '{}', STU_QUALIFICATION = '{}' WHERE STU_ID = '{}'".format(
            UP_name.value, UP_address.value, UP_email.value, UP_currentJob.value, UP_qualification.value, UP_id.value
        ))
        c.execute("commit")
      except:
        UP_status_label.value = "<span style='color: red'>An error has occured</span>"
      else:
        UP_status_label.value = "<span style='color: green'>Student ID("+UP_id.value+") updated!</span>"
    UP_update_button.on_click(on_UP_update_button_clicked)

    UP_line1HBox = widgets.HBox([UP_id, UP_name])
    UP_line2HBox = widgets.HBox([UP_address, UP_email])
    UP_line3HBox = widgets.HBox([UP_currentJob, UP_qualification])
    UP_line4HBox = widgets.HBox([UP_status_label])
    UP_line5HBox = widgets.HBox([UP_update_button])

    display(widgets.VBox([UP_line1HBox, UP_line2HBox, UP_line3HBox, UP_line4HBox, UP_line5HBox]))
################ Student Main Tab - End ################

STU_tabGroup1()